In [5]:
import pandas as pd

df = pd.read_pickle('../data/social_explorer/combined_df.pkl')

In [6]:
pd.options.display.max_rows = 4000

In [11]:
df.sample(5)['Combined'].values

array(['[INST] What are the key terrain characteristics of Teton County? [/INST] Teton County in Montana has the following terrain makeup: 0.702494801% open water, 0.0% perennial ice/snow, 1.392193262% developed open space, 0.293982263% developed low intensity, 0.015479932% developed medium intensity, 0.0031536% developed high intensity, 2.525806457% barren land (rock/sand/clay), 0.030035011% deciduous forest, 10.81043587% evergreen forest, 0.36590861% mixed forest, 9.436679041% shrub/scrub, 36.67872211% grassland/herbaceous, 2.782991663% pasture/hay, 32.94126601% cultivated crops, 1.607517455% woody wetlands, 0.413333907% emergent herbaceous wetlands, 35.72425767% farmland, 1.686175525% houses with lots of land, 0.015479932% suburbia, 0.0031536% big city, 11.20637949% forests, 46.11540115% open fields, 2.020851362% wetlands, 0.702494801% open water, 2.525806457% rock/sand/clay, 0.0% perennial ice/snow. </s>',
       '[INST] What are the key terrain characteristics of Medina County? [/

In [13]:
import re

def split_combined(text):
    pattern = r"\[INST\](.*?)\[/INST\](.*)"
    match = re.match(pattern, text, re.DOTALL)
    
    if match:
        prompt = match.group(1).strip()
        answer = match.group(2).strip()
        return prompt, answer
    else:
        return None, None

df[['prompt', 'answer']] = df['Combined'].apply(lambda x: pd.Series(split_combined(x)))

print(df[['prompt', 'answer']].head())


                                              prompt  \
0  What are the key terrain characteristics of Do...   
1  What are the key terrain characteristics of Fo...   
2  What are the key terrain characteristics of Ma...   
3  What are the key terrain characteristics of Tr...   
4  What are the key terrain characteristics of Wa...   

                                              answer  
0  Douglas County in Illinois has the following t...  
1  Ford County in Illinois has the following terr...  
2  Marshall County in Illinois has the following ...  
3  Traverse County in Minnesota has the following...  
4  Watonwan County in Minnesota has the following...  


In [14]:
def create_dialog_row(row):
    return {
        "dialog": [
            {"content": row['prompt'], "role": "user"},
            {"content": row['answer'], "role": "assistant"}
        ]
    }

df['dialog'] = df.apply(create_dialog_row, axis=1)

df = df.drop(columns=['Combined', 'prompt', 'answer'])

df.head()

,dialog
0,{'dialog': [{'content': 'What are the key terr...
1,{'dialog': [{'content': 'What are the key terr...
2,{'dialog': [{'content': 'What are the key terr...
3,{'dialog': [{'content': 'What are the key terr...
4,{'dialog': [{'content': 'What are the key terr...


In [18]:
df['dialog'].to_json('../data/social_explorer/formatted_dialog.jsonl', index=False, orient='records', lines=True)

In [19]:
import sagemaker

sagemaker_session = sagemaker.Session()

sagemaker_session.upload_data('../data/social_explorer/formatted_dialog.jsonl')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/kevin/.config/sagemaker/config.yaml


's3://sagemaker-us-east-1-702497914099/data/formatted_dialog.jsonl'